# Classer ses iris (KNN)

David Scanu 

* v3 - Feature Selection

---

## Contexte du projet

Une chaine de fleuriste aimerait pouvoir trier ses différentes espèces d'iris.
Réalisez un programme permettant de prédire l'espèce d'une iris à partir de la largeur et longueur de ses sépales et des pétales.

## Modalités pédagogiques

•	Travail individuel
•	deux jours de travail

## Critères de performance

•	Les données ont été analysées et il existe une trace de cette analyse exploratoire dans un jupyter-notebook
•	Un programme qui fonctionne sans bug, et qui classifie bien les iris (vous afficherez la matrice de confusion et l'accuracy obtenus sur la base Test)

## Modalités d'évaluation

Revue du code avec le formateur

## Livrables

Dépot Github

---

## Importer les bibliothèques

In [237]:
from sklearn import datasets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# import some data to play with
iris = datasets.load_iris() # nd.array

### Quel est notre Objectif ?

Compte tenu de : 
- **longueur des sépales**
- **largeur des sépales**
- **longueur des pétales**
- **largeur des pétales**
  
Classez la fleur d'iris dans l'une des trois espèces - **Setosa**, **Virginica** et **Versicolor**.

## Convertir le nd.array en DataFrame

In [250]:
# np.c_ is the numpy concatenate function
# which is used to concat iris['data'] and iris['target'] arrays 
# for pandas column argument: concat iris['feature_names'] list
# and string list (in this case one string); you can make this anything you'd like..  
# the original dataset would probably call this ['Species']
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [251]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [252]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [240]:
# Preparation du dataset
# Convert target numbers to "Iris Species"
def chng(target):
    if target == 0:
        return 'Setosa'
    elif target == 1:
        return 'Versicolor'
    elif target == 2:
        return 'Virginica'

iris_df['target'] = iris_df['target'].apply(chng)

In [ ]:
# Renaming columns 'target'  to 'species'ArithmeticError
iris_df = iris_df.rename(columns={"target": "species"})

In [ ]:
print(iris_df.head())

## Analyse statistique de base

### Moyennes et médianes

In [ ]:
iris_df.groupby('species').agg(['mean', 'median'])

Pour chaque caractère, la moyenne et la médiane sont relativement proches, ce qui indique une distribtuion normal de la population pour chaque caractère.

### Ecart-type par espèces

In [ ]:
iris_df.groupby('species').std()

### Boxplot

In [ ]:
sns.set(style="ticks") 
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
sns.boxplot(x='species',y='sepal length (cm)',data=iris_df)
plt.subplot(2,2,2)
sns.boxplot(x='species',y='sepal width (cm)',data=iris_df)
plt.subplot(2,2,3)
sns.boxplot(x='species',y='petal length (cm)',data=iris_df)
plt.subplot(2,2,4)
sns.boxplot(x='species',y='petal width (cm)',data=iris_df)
plt.show()

Les points isolés qui peuvent être vus dans les boîtes à moustaches ci-dessus sont les valeurs aberrantes dans les données. Comme ceux-ci sont très peu nombreux, cela n'aurait pas d'impact significatif sur notre analyse.

### Violin Plot

Montre la distribution des données sur plusieurs niveaux d'une (ou plusieurs) variables catégorielles (espèces de fleurs dans notre cas) de sorte que ces distributions puissent être comparées.

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
sns.violinplot(x='species', y='sepal length (cm)',data=iris_df)
plt.subplot(2,2,2)
sns.violinplot(x='species',y='sepal width (cm)',data=iris_df)
plt.subplot(2,2,3)
sns.violinplot(x='species',y='petal length (cm)',data=iris_df)
plt.subplot(2,2,4)
sns.violinplot(x='species',y='petal width (cm)',data=iris_df)
plt.show()

### Pairplot

Premier reflexe, tracer un pairplot pour mettre en évidence répartitions et corrélations.

Grace au pairplot, nous pouvons mettre en évidence les caractères qui sont correlés et qui sont pertinents pour determiner la vartiété d'Iris.

In [ ]:
sns.pairplot(data=iris_df, hue="species", height=3)

### Analyse univariée

Probability Density Function (PDF) & Cumulative Distribution Function (CDF)

In [ ]:
iris_setosa = iris_df[iris_df['species'] == "Setosa"]
iris_versicolor = iris_df[iris_df['species'] == "Versicolor"]
iris_virginica = iris_df[iris_df['species'] == "Virginica"]

In [ ]:
# sepal length
sns.FacetGrid(iris_df, hue="species", height=5).map(sns.histplot, "sepal length (cm)").add_legend();
# sepal width
sns.FacetGrid(iris_df, hue="species", height=5).map(sns.histplot, "sepal width (cm)").add_legend();
# petal length
sns.FacetGrid(iris_df, hue="species", height=5).map(sns.histplot, "petal length (cm)").add_legend();
# petal width
sns.FacetGrid(iris_df, hue="species", height=5).map(sns.histplot, "petal width (cm)").add_legend();
plt.show()

Sur le graphique 1 et 2, les répartitions se chevauchent beaucoup et ont ne peu pas réellement en déduire que la variable est determinante comme indication de la variété.

Sur le graphique 3, la densité de la longueur de "**petal length**" semble prometteur du point de vue de la classification univariée. Les espèces **Setosa** sont bien séparées de **Versicolor** et **Virginica**, bien qu'il y ait un certain chevauchement entre **Versicolor** et **Virginica**.

Sur le graphique 4, le tracé de densité de 'petal width' semble également bon. Il y a une légère intersection entre les espèces **Setosa** et **Versicolor**, tandis que le chevauchement entre **Versicolor** et **Virginica** est quelque peu similaire à celui de la longueur des pétales (Graphique 3).

Pour résumer, si nous devons choisir une caractéristique pour la classification, nous choisirons la **"petal length"** (Graphique 3) pour distinguer les espèces.

Si nous devons sélectionner deux caractéristiques, nous choisirons **'petal width'** comme deuxième caractéristique, mais encore une fois, il serait plus sage d'examiner les graphiques en paires (analyse bivariée et multivariée) pour déterminer quelles sont les deux caractéristiques les plus utiles dans classification.

### Analyse Bivariée

In [ ]:
# Creating a DataFrame with the ndarray from load_iris()
iris_enc_df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])

#### Tableau de correlations

In [ ]:
# Afficher les correlations
iris_enc_df.corr()

#### Heatmap

In [ ]:
sns.heatmap(iris_enc_df.corr(), annot=True, cmap='RdBu')

On distingue une forte correlation entre la variété d'Iris et Petal Length et Petal Width. Ces deux criters seuls suffiraient à réaliser des prédictions.

#### Scatter Plot

In [ ]:
sns.scatterplot(data=iris_df, x='sepal length (cm)', y='sepal width (cm)', hue='species')
plt.title('Sepal width by Sepal length')
plt.show()

In [ ]:
sns.scatterplot(data=iris_df, x='petal length (cm)', y='petal width (cm)', hue='species')
plt.title('Petal Width by Petal Length')
plt.show()

On distingue bien sur ce graphique, que les elements sont répartis sur un ligne et que le caractère 'petal width' et 'petal length' permettent de discriminer la variété d'iris.

## Feature Selection

In [ ]:
# Features
X = iris.data

# Target
y = iris.target

#### VarianceThreshold

Elimine les variables dont la **variance est inférieur à un certain seuil**.

In [ ]:
# Variance des variables
X.var(axis=0)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector_vt = VarianceThreshold(threshold=0.2)
selector_vt.fit(X)
# Affiche un masque
selector_vt.get_support()


In [ ]:
# Affiche les colonnes restantes
np.array(iris.feature_names)[selector_vt.get_support()]

In [ ]:
X_vt = selector_vt.transform(X)
X_vt[:10,:] # Affiche les 10 première ligne de notre ndarray

#### SelectKbest

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Retourne 2 tableaux :
# - score test chi2, dépendance à y
# - P values
chi2(X, y)

In [ ]:
selector_kb = SelectKBest(chi2, k=2)
selector_kb.fit(X, y)
selector_kb.get_support() # Retourne une seule variable/colonne

In [242]:
# Affiche les colonnes restantes
np.array(iris.feature_names)[selector_kb.get_support()]

array(['petal length (cm)', 'petal width (cm)'], dtype='<U17')

In [244]:
X_col_names = list(np.array(iris.feature_names)[selector_kb.get_support()])

In [245]:
X_col_names

['petal length (cm)', 'petal width (cm)']

In [241]:
X_kb = selector_kb.transform(X)
X_kb[:10] # Affiche les 10 première ligne de notre ndarray

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

#### SelectFromModel

Entraine un estimateur puis selectionne les **variables** les plus importantes pour cet estimateur. Compatible avec les estimateurs qui développent une fonction paramétrée *(Ne fonctionne pas avec Knn)*.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import SGDClassifier

selector_sfm = SelectFromModel(SGDClassifier(random_state=0), threshold='mean')
selector_sfm.fit(X, y)
selector_sfm.get_support()

In [ ]:
# Affiche les colonnes restantes
np.array(iris.feature_names)[selector_sfm.get_support()]

In [ ]:
X_sfm = selector_sfm.transform(X)
X_sfm[:10]

In [ ]:
# Matrice coefficient de 3 x 4
selector_sfm.estimator_.coef_

In [ ]:
# Le selecteur selectionne les variables > à ce chiffre
selector_sfm.estimator_.coef_.mean(axis=0).mean()

#### RFE + RFECV

Eliminent les variables les moins importantes de façon **récursive**.

Un estimateur est entrainé plusieurs fois, après chaque entrainement, des features sont éliminées sur la base de **coefficients** les plus faibles de l'estimateur.



In [ ]:
from sklearn.feature_selection import RFE, RFECV

selector_RFECV = RFECV(SGDClassifier(random_state=0), step=1, min_features_to_select=2, cv=5)
selector_RFECV.fit(X, y)
selector_RFECV.ranking_


In [ ]:
selector_RFECV.cv_results_

In [ ]:
selector_RFECV.get_support()

In [ ]:
# Affiche les colonnes restantes
np.array(iris.feature_names)[selector_RFECV.get_support()]

In [ ]:
X_RFECV = selector_RFECV.transform(X)
X_RFECV[:10]

### Préparations des données

#### Nettoyage

il n'y a pas de valeurs manquantes dans le jeu de données.

In [ ]:
# Y-a-t il des valuers manquantes ?
print(iris_df.isnull().values.any())

In [ ]:
print(iris_df.isnull().sum())

#### Outliers

In [ ]:
# Visualisation des outliers avec boxplot()
sns.set(style="ticks") 
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
sns.boxplot(x='species',y='sepal length (cm)',data=iris_df)
plt.subplot(2,2,2)
sns.boxplot(x='species',y='sepal width (cm)',data=iris_df)
plt.subplot(2,2,3)
sns.boxplot(x='species',y='petal length (cm)',data=iris_df)
plt.subplot(2,2,4)
sns.boxplot(x='species',y='petal width (cm)',data=iris_df)
plt.show()

Il y a très peu d'**outliers**. Ils ne seront pas génants pour l'entrainement de notre modèle.

#### Equilibre

Les cibles du jeu de données sont **bien équilibrées** puisqu'elles sont au nombre de 50 chacunes.

In [ ]:
iris_df['species'].value_counts()

### Encoder les données

Les données contenues dans le **ndarray** récupérées avec **load_iris()** sont toutes numériques et ne nécessitent pas d'encodage.

### Séparer les données

In [ ]:
from sklearn.model_selection import train_test_split

# Features
X_train, X_test, y_train, y_test = train_test_split(X_kb, y, test_size=0.2, random_state=3)

print(f"Nombre d'exemples d'entrainement X : {X_train.shape[0]}")
print(f"Nombre d'exemples de test X : {X_test.shape[0]}")
print(' ')
print(f"Nombre d'exemples d'entrainement Y : {y_train.shape[0]}")
print(f"Nombre d'exemples de test Y : {y_test.shape[0]}")

### Mise à l'échelle

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train_minmaxscaler = min_max_scaler.fit_transform(X_train)
X_test_minmaxscaler = min_max_scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()
X_train_robscaler = robust_scaler.fit_transform(X_train)
X_test_robscaler = robust_scaler.transform(X_test)

### KNeighborsClassifier

#### Modèle KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Nous entrainons notre modèle avec les données d'entrainements (standardisées)
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train_scaler, y_train)
print('Train Score : ', model.score(X_train_scaler, y_train))
print('Test Score : ', model.score(X_test_scaler, y_test))

#### Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, LeaveOneOut, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit
from numpy import mean

# Choix de la méthode de validation croisée (échantillonage)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
# cv = LeaveOneOut()

cross_val_res = cross_val_score(model, X_train_scaler, y_train, cv=cv, scoring='accuracy')
print(cross_val_res)

In [ ]:
cross_val_res.mean()

In [ ]:
# Cross-validation for each k value
val_score = []
for k in range(1, 50):
    score = cross_val_score(KNeighborsClassifier(n_neighbors=k), X_train_scaler, y_train, cv=5).mean()
    val_score.append(score)
plt.plot(val_score)

In [ ]:
def cv_choice(cv_list,  X_train, y_train, k_range = range(1, 20)):
    """Find the best cross-validation about our problems and model: K-Fold, Leave One Out, Shuffle Split, Stratified K-Fold, Group K-Fold
    Return best CV, Max Validation Score and optimal K value. 
    """
    
    print("K range : ", k_range)

    cv_results = {}

    cv_retenu=None
    max_val_score=0
    k_retenu=None

    for cv in cv_list:
        cv_name = str(cv).partition("(")[0]
        print(cv_name)
        cv_scores = []
        for k in k_range:
            score = cross_val_score(KNeighborsClassifier(n_neighbors=k), X_train, y_train, cv=cv, scoring='accuracy').mean()
            # print(f"{cv_name} score : {score}")
            cv_scores.append(score)

        # Plot the graph
        plt.plot(cv_scores, label=cv_name)

        # Print
        cv_scores_max = max(cv_scores)
        cv_scores_min = min(cv_scores)
        cv_scores_mean = mean(cv_scores)
        cv_results[cv] = {'mean' : cv_scores_mean, 'min' : cv_scores_min, 'max' : cv_scores_max}
        print(f"- CV name : {cv_name} - CV mean : {cv_scores_mean} - CV min : {cv_scores_mean} - CV max : {cv_scores_max}")

        # Saving values
        k_val=k_range[np.argmax(np.array(cv_scores))]
        if cv_retenu==None or max_val_score<cv_scores_max :
            cv_retenu=cv
            max_val_score=cv_scores_max
            k_retenu=k_val

    # Show the graph
    plt.legend()
    plt.show()

    # print(cv_results)
    print("cv retenu : ", cv_retenu)
    print("max_val_score : ", max_val_score)
    print("k_retenu : ", k_retenu)

    return cv_retenu, max_val_score, k_retenu

In [ ]:
cv_list = [KFold(n_splits=4), LeaveOneOut(), ShuffleSplit(n_splits=4, train_size=0.8), StratifiedKFold(n_splits=4, shuffle=True)]

# Chossing the best Cross-validation method
cv_choice(cv_list, X_train_scaler, y_train)

#### Validation Curve

In [ ]:
from sklearn.model_selection import validation_curve

k = np.arange(1, 50)
train_score, val_score = validation_curve(KNeighborsClassifier(), X_train_scaler, y_train, param_name='n_neighbors', param_range=k, cv=5)

plt.plot(k, val_score.mean(axis=1), label="Validation")
plt.plot(k, train_score.mean(axis=1), label="Train")
plt.ylabel('score')
plt.xlabel('n_neighbors')
plt.legend()
plt.show()

#### Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors' : np.arange(1, 20), 'metric' : ['euclidean', 'manhattan']}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train_scaler, y_train)

Afficher le meilleur score

In [ ]:
grid.best_score_

Afficher les paramètres donnant les meilleures performances

In [ ]:
grid.best_params_

Sauvegarder le meilleur modèle

In [ ]:
# Best model
model_best = grid.best_estimator_

Tester le nouveau modèle sur les données de test

In [ ]:
# Score du meilleur modèle
model_best.score(X_test_scaler, y_test)

#### Grid Search CV Function

In [233]:
def grid_choice(model, param_grid, cv_list, X_train, y_train, X_test, y_test, k_range = range(1, 20)):

    best_model = None
    best_params = None
    best_cv = None
    best_cv_score = 0

    for cv in cv_list:

        cv_name = str(cv).partition("(")[0]
        print(cv_name)

        grid = GridSearchCV(model, param_grid, cv=cv)
        grid.fit(X_train, y_train)

        # CV
        grid_cv_score = grid.best_score_

        print(f"CV name : {cv_name} - CV score : {grid_cv_score}")

        # Saving values
        if best_model==None or best_cv_score<grid_cv_score :
            best_cv_score = grid_cv_score
            best_model = grid.best_estimator_
            best_params = grid.best_params_
            best_cv = cv
            best_model_test_score = grid.best_estimator_.score(X_test, y_test)

    return best_model, best_params, best_model_test_score, best_cv


In [234]:
best_model, best_params, best_model_test_score, best_cv = grid_choice(KNeighborsClassifier(), {'n_neighbors' : np.arange(1, 20), 'metric' : ['euclidean', 'manhattan']}, cv_list, X_train_scaler, y_train, X_test_scaler, y_test)

print("Best model : ", best_model)
print("Best params : ", best_params)
print("Best CV : ", best_cv)
print("Best model test score : ", best_model_test_score)

KFold
CV name : KFold - CV score : 0.9583333333333333
LeaveOneOut
CV name : LeaveOneOut - CV score : 0.9583333333333334
ShuffleSplit
CV name : ShuffleSplit - CV score : 0.96875
StratifiedKFold
CV name : StratifiedKFold - CV score : 0.9666666666666667
Best model :  KNeighborsClassifier(metric='euclidean', n_neighbors=4)
Best params :  {'metric': 'euclidean', 'n_neighbors': 4}
Best CV :  ShuffleSplit(n_splits=4, random_state=None, test_size=None, train_size=0.8)
Best model test score :  1.0


### Matrice de confusion

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model_best.predict(X_test_scaler)
confusion_matrix = confusion_matrix(y_test, y_pred)


In [ ]:
data = {'prediction': y_pred, 'actual': y_test}
df = pd.DataFrame(data)
contingency_matrix = pd.crosstab(df['prediction'], df['actual'])
print(contingency_matrix)

In [ ]:
sns.heatmap(contingency_matrix.T, annot=True, fmt='.2f', cmap="YlGnBu", cbar=False)
plt.title('Matrice de confusion')
plt.show()

#### Learning Curve

In [ ]:
from sklearn.model_selection import learning_curve

N, train_score, val_score = learning_curve(model_best, X_train_scaler, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

print(N)
plt.plot(N, train_score.mean(axis=1), label='train')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()
plt.show()

## Export du modèle

Exporter le modèle avec joblib ou Pickle. Il faut exporter :
- Le modèle
- Le scaler
- le nom des colonnes X
- les valeurs possibles de la 'target' (si catégorie)

In [254]:
import pickle

dict_export = {}
dict_export['model'] = model_best
dict_export['scaler'] = scaler
dict_export['X_col_names'] = X_col_names
dict_export['y_names'] = list(iris.target_names)

pickle_out = open("model.pkl","wb")
pickle.dump(dict_export, pickle_out)
pickle_out.close()